In [1]:
# from PIL import Image
from osgeo import gdal,ogr, osr
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import os

from matplotlib import pyplot as plt

from sklearn.preprocessing import PowerTransformer,MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import imblearn
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
from numpy import mean
from numpy import std
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
import pandas as pd
import re
import pickle
from os import listdir
from os.path import isfile, join

2022-09-06 02:59:06.959809: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-06 02:59:06.959842: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
# Intrested bands
BANDS = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'longitude', 'latitude', 'constant', 
         'NDVI', 'EVI', 'NDVI_1', 'EVI_1', 'NDVI_2', 'EVI_2', 'NDVI_3', 'EVI_3', 'NDVI_4', 'EVI_4', 
         'NDVI_5', 'EVI_5', 'NDVI_6', 'EVI_6', 'NDVI_7', 'EVI_7', 'NDVI_8', 'EVI_8', 'NDVI_9', 'EVI_9', 
         'NDVI_10', 'EVI_10', 'cultivated', 'label']
print(len(BANDS))

33


In [10]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [11]:
my_path = "/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/"
onlyfiles = [f for f in listdir(my_path)]
print(onlyfiles)

['complete_2013-0000017664-0000000000.tif', 'complete_2013-0000011776-0000005888-013.tif', 'complete_2013-0000017664-0000005888.tif', 'complete_2013-0000000000-0000000000-005.tif', 'complete_2013-0000005888-0000000000-007.tif', 'complete_2013-0000005888-0000011776.tif', 'complete_2013-0000000000-0000005888-017.tif', 'complete_2013-0000000000-0000011776.tif', 'complete_2013-0000011776-0000000000-016.tif', 'complete_2013-0000005888-0000005888-001.tif', 'complete_2013-0000017664-0000011776.tif', 'complete_2013-0000011776-0000011776.tif']


In [12]:
# '''
# Numpy Crop 3D Array to Non-NaN Values
# Crops value region based on labels" ignoring nans"
# Input: image array with nans features*row_col
# Output: image array with nans features*row*col
# ''' 
# def crop_nparray(complete_image):
    
# #     print("lol",complete_image.shape)
#     lbl = complete_image[-1]
# #     print(lbl)
# #     (unique_2, counts_2) = np.unique(lbl, return_counts=True)    
# #     print(unique_2, counts_2)
#     nans = np.isnan(lbl)
#     nancols = np.all(nans, axis=0) 
#     nanrows = np.all(nans, axis=1) 

#     firstcol = nancols.argmin() 
#     firstrow = nanrows.argmin() 

#     lastcol = len(nancols) - nancols[::-1].argmin() 
#     lastrow = len(nanrows) - nanrows[::-1].argmin() 
# #     print("first", firstrow, firstcol)
# #     print("last", lastrow, lastcol)
# #     print(lbl[firstrow:lastrow,firstcol:lastcol].shape)
#     cropped_image = complete_image[:, firstrow:lastrow,firstcol:lastcol]
# #         print(cropped_image.shape)
#     return cropped_image

In [13]:
'''
Generate Image and Lables from tif using gdal
Input: list of file path by year [[]]
output: returns a pandas data frame in the format of a csv
'''
def generate_image_labels(list_files, mypath, BANDS):
    df = pd.DataFrame()
    yearly_image = []
#     print("list_files", list_files)
    for file in list_files:
        dtype='float32'
        cropped_images = []
        each_file=(os.path.join(mypath,file))
        print(each_file)
        dataset_bands = gdal.Open(each_file)
        complete_image = np.array([dataset_bands.GetRasterBand(i).ReadAsArray() for i in range(1, dataset_bands.RasterCount + 1)]).astype(dtype)
        df2 = pd.DataFrame()
        for i in range(dataset_bands.RasterCount):
            df2[BANDS[i]] = complete_image[i].ravel()

        df2 = df2.drop(columns=['constant'])
        df2 = df2.dropna()
        df = pd.concat([df, df2])

    return df

In [14]:
yearly_df = generate_image_labels(onlyfiles, my_path, BANDS)

/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000017664-0000000000.tif
/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000011776-0000005888-013.tif
/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000017664-0000005888.tif
/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000000000-0000000000-005.tif
/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000005888-0000000000-007.tif
/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000005888-0000011776.tif
/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000000000-0000005888-017.tif
/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000000000-0000011776.tif
/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000011776-0000000000-016.tif
/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000005888-0000005888-001.tif
/raid/Praveen_Stuff/Madison/NEW_Cultivated_2013/complete_2013-0000017664-0000011776.tif
/raid/Pr

In [16]:
yearly_df.head()

,BLUE,GREEN,RED,NIR,SWIR1,SWIR2,longitude,latitude,NDVI,EVI,...,NDVI_7,EVI_7,NDVI_8,EVI_8,NDVI_9,EVI_9,NDVI_10,EVI_10,cultivated,label
1947,0.027067,0.058692,0.060452,0.164347,0.137287,0.081435,-87.948616,30.247667,0.5747,0.2903,...,0.8016,0.3294,0.5878,0.2674,0.7035,0.2802,0.638,0.3296,0.0,0.0
1948,0.025665,0.055585,0.056355,0.159782,0.128707,0.075165,-87.948341,30.247667,0.5747,0.2903,...,0.8016,0.3294,0.5878,0.2674,0.7035,0.2802,0.638,0.3296,0.0,0.0
1949,0.024593,0.056823,0.055942,0.170178,0.132722,0.075770,-87.948074,30.247667,0.5747,0.2903,...,0.8016,0.3294,0.5878,0.2674,0.7035,0.2802,0.638,0.3296,0.0,0.0
1950,0.025308,0.058252,0.055888,0.171717,0.134400,0.075165,-87.947807,30.247667,0.5747,0.2903,...,0.8016,0.3294,0.5878,0.2674,0.7035,0.2802,0.638,0.3296,0.0,0.0
1951,0.027040,0.059958,0.057125,0.168830,0.135830,0.076980,-87.947540,30.247667,0.5747,0.2903,...,0.8016,0.3294,0.5878,0.2674,0.7035,0.2802,0.638,0.3296,0.0,0.0


In [17]:
# Save that csv file which is the complete data
yearly_df.to_csv("/raid/Praveen_Stuff/Madison/Cultivated_2013.csv") 

In [8]:
path = "/raid/Praveen_Stuff/Madison/CSV/Complete_2017.csv"
df = pd.read_csv(path)

(175006525, 33)


In [ ]:
'''
Generate cultivated dataframe using pandas
Input: Complete Dataframe for the specific year
output: returns a pandas data frame in the format of a csv for cultivated region
'''
def create_cultivated_set(dataframe):
    # Create a set based on where the cultivated column values is 1 use this set in training
    # return that dataframe
    df_new = dataframe[dataframe['cultivated'] == 1.0]
    df_new = df_new.drop(columns=['cultivated'])
    return df_new
'''
Generate cultivated dataframe using pandas
Input: Complete Dataframe for the specific year
output: returns a pandas data frame in the format of a csv for non-cultivated region
'''
def create_non_cultivated_set(dataframe):
    # Create a set based on where the cultivated column values is 0, use this set in testing
    # return that dataframe
    df_new = dataframe[dataframe['cultivated'] == 0.0]
    df_new = df_new.drop(columns=['cultivated'])
    return df_new

In [ ]:
cultivated_df = create_cultivated_set(df)
non_cultivated_df = create_non_cultivated_set(df)

In [ ]:
# Save Cultivated and non-cultivated df
cultivated_df.to_csv("/raid/Praveen_Stuff/Madison/CSV/cultivated_2017.csv") 
non_cultivated_df.to_csv("/raid/Praveen_Stuff/Madison/CSV/non_cultivated_2017.csv") 